In [ ]:
#Install BioPython
!pip install Bio

In [ ]:
#@title Input protein sequence, jobaname and modify alignment paramters if relevant
from google.colab import files
import os
import os.path
import re
import hashlib

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

s = 'MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGESGSGKSVLTKTFTG MLEENGRIAQGSIDYRGQDLTALSSHKDWEQIRGAKIATIFQDPMTSLDPIKTIGSQITE VIVKHQGKTAKEAKELAIDYMNKVGIPDADRRFNEYPFQYSGGMRQRIVIAIALACRPDV LICDEPTTALDVTIQAQIIDLLKSLQNEYHFTTIFITHDLGVVASIADKVAVMYAGEIVE YGTVEEVFYDPRHPYTWSLLSSLPQLADDKGDLYSIPGTPPSLYTDLKGDAFALRSDYAM QIDFEQKAPQFSVSETHWAKTWLLHEDAPKVEKPAVIANLHDKIREKMGFAHLAD' #@param {type:"string"}
# remove whitespaces
query_sequence = "".join(s.split())
query_sequence = re.sub(r'[^a-zA-Z]','', query_sequence).upper()

jobname = 'AMIE' #@param {type:"string"}
# remove whitespaces
jobname = "".join(jobname.split())
jobname = re.sub(r'\W+', '', jobname)
jobname = add_hash(jobname, query_sequence)


with open(f"{jobname}.fasta", "w") as text_file:
    text_file.write(">1\n%s" % query_sequence)

# number of models to use
#@markdown ---
#@markdown ### Advanced settings
msa_mode = "MMseqs2 (UniRef only)" #@param ["MMseqs2 (UniRef+Environmental)", "MMseqs2 (UniRef only)","single_sequence","custom"]
num_models = 5 #@param [1,2,3,4,5] {type:"raw"}
use_msa = True if msa_mode.startswith("MMseqs2") else False
use_env = True if msa_mode == "MMseqs2 (UniRef+Environmental)" else False
use_custom_msa = True if msa_mode == "custom" else False
use_amber = False #@param {type:"boolean"}
use_templates = False #@param {type:"boolean"}
#@markdown ---
#@markdown ### Experimental options
homooligomer = 1 #@param [1,2,3,4,5,6,7,8] {type:"raw"}
save_to_google_drive = False #@param {type:"boolean"}
#@markdown ---
#@markdown Don't forget to hit `Runtime` -> `Run all` after updating the form.


if homooligomer > 1:
  if use_amber:
    print("amber disabled: amber is not currently supported for homooligomers")
    use_amber = False
  if use_templates:
    print("templates disabled: templates are not currently supported for homooligomers")
    use_templates = False

with open(f"{jobname}.log", "w") as text_file:
    text_file.write("num_models=%s\n" % num_models)
    text_file.write("use_amber=%s\n" % use_amber)
    text_file.write("use_msa=%s\n" % use_msa)
    text_file.write("msa_mode=%s\n" % msa_mode)
    text_file.write("use_templates=%s\n" % use_templates)
    text_file.write("homooligomer=%s\n" % homooligomer)

# decide which a3m to use
if use_msa:
  a3m_file = f"{jobname}.a3m"
elif use_custom_msa:
  a3m_file = f"{jobname}.custom.a3m"
  if not os.path.isfile(a3m_file):
    custom_msa_dict = files.upload()
    custom_msa = list(custom_msa_dict.keys())[0]
    header = 0
    import fileinput
    for line in fileinput.FileInput(custom_msa,inplace=1):
      if line.startswith(">"):
         header = header + 1 
      if line.startswith("#"):
        continue
      if line.rstrip() == False:
        continue
      if line.startswith(">") == False and header == 1:
         query_sequence = line.rstrip() 
      print(line, end='')

    os.rename(custom_msa, a3m_file)
    print(f"moving {custom_msa} to {a3m_file}")
else:
  a3m_file = f"{jobname}.single_sequence.a3m"
  with open(a3m_file, "w") as text_file:
    text_file.write(">1\n%s" % query_sequence)

if save_to_google_drive == True:
  from pydrive.drive import GoogleDrive
  from pydrive.auth import GoogleAuth
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("You are logged into Google Drive and are good to go!")

In [ ]:
#@title Install dependencies
%%bash -s $use_amber $use_msa $use_templates

USE_AMBER=$1
USE_MSA=$2
USE_TEMPLATES=$3

if [ ! -f AF2_READY ]; then
  # install dependencies
  pip -q install biopython dm-haiku ml-collections py3Dmol
  wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/colabfold.py

  # download model
  if [ ! -d "alphafold/" ]; then
    git clone https://github.com/deepmind/alphafold.git --quiet
    (cd alphafold; git checkout 1d43aaff941c84dc56311076b58795797e49107b --quiet)
    mv alphafold alphafold_
    mv alphafold_/alphafold .
    # remove "END" from PDBs, otherwise biopython complains
    sed -i "s/pdb_lines.append('END')//" /content/alphafold/common/protein.py
    sed -i "s/pdb_lines.append('ENDMDL')//" /content/alphafold/common/protein.py
  fi

  # download model params (~1 min)
  if [ ! -d "params/" ]; then
    mkdir params
    curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar \
    | tar x -C params
  fi
  touch AF2_READY
fi
# download libraries for interfacing with MMseqs2 API
if [ ${USE_MSA} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f MMSEQ2_READY ]; then
    apt-get -qq -y update 2>&1 1>/dev/null
    apt-get -qq -y install jq curl zlib1g gawk 2>&1 1>/dev/null
    touch MMSEQ2_READY
  fi
fi
# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign3=3.2.2 hhsuite=3.3.0 python=3.7 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=7.5.1 python=3.7 pdbfixer 2>&1 1>/dev/null
  (cd /usr/local/lib/python3.7/site-packages; patch -s -p0 < /content/alphafold_/docker/openmm.patch)
  wget -qnc https://git.scicore.unibas.ch/schwede/openstructure/-/raw/7102c63615b64735c4941278d92b554ec94415f8/modules/mol/alg/src/stereo_chemical_props.txt
  mv stereo_chemical_props.txt alphafold/common/
  touch AMBER_READY
fi

In [ ]:
#@title Import libraries
# setup the model
if "model" not in dir():

  # hiding warning messages
  import warnings
  from absl import logging
  import os
  import tensorflow as tf
  warnings.filterwarnings('ignore')
  logging.set_verbosity("error")
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  tf.get_logger().setLevel('ERROR')

  import sys
  import numpy as np
  import pickle
  from alphafold.common import protein
  from alphafold.data import pipeline
  from alphafold.data import templates
  from alphafold.model import data
  from alphafold.model import config
  from alphafold.model import model
  from alphafold.data.tools import hhsearch
  import colabfold as cf

  # plotting libraries
  import py3Dmol
  import matplotlib.pyplot as plt
  import ipywidgets
  from ipywidgets import interact, fixed, GridspecLayout, Output


if use_amber and "relax" not in dir():
  sys.path.insert(0, '/usr/local/lib/python3.7/site-packages/')
  from alphafold.relax import relax

def mk_mock_template(query_sequence):
  # since alphafold's model requires a template input
  # we create a blank example w/ zero input, confidence -1
  ln = len(query_sequence)
  output_templates_sequence = "-"*ln
  output_confidence_scores = np.full(ln,-1)
  templates_all_atom_positions = np.zeros((ln, templates.residue_constants.atom_type_num, 3))
  templates_all_atom_masks = np.zeros((ln, templates.residue_constants.atom_type_num))
  templates_aatype = templates.residue_constants.sequence_to_onehot(output_templates_sequence,
                                                                    templates.residue_constants.HHBLITS_AA_TO_ID)
  template_features = {'template_all_atom_positions': templates_all_atom_positions[None],
                       'template_all_atom_masks': templates_all_atom_masks[None],
                       'template_sequence': [f'none'.encode()],
                       'template_aatype': np.array(templates_aatype)[None],
                       'template_confidence_scores': output_confidence_scores[None],
                       'template_domain_names': [f'none'.encode()],
                       'template_release_date': [f'none'.encode()]}
  return template_features

def mk_template(a3m_lines, template_paths):
  template_featurizer = templates.TemplateHitFeaturizer(
      mmcif_dir=template_paths,
      max_template_date="2100-01-01",
      max_hits=20,
      kalign_binary_path="kalign",
      release_dates_path=None,
      obsolete_pdbs_path=None)

  hhsearch_pdb70_runner = hhsearch.HHSearch(binary_path="hhsearch", databases=[f"{template_paths}/pdb70"])

  hhsearch_result = hhsearch_pdb70_runner.query(a3m_lines)
  hhsearch_hits = pipeline.parsers.parse_hhr(hhsearch_result)
  templates_result = template_featurizer.get_templates(query_sequence=query_sequence,
                                                       query_pdb_code=None,
                                                       query_release_date=None,
                                                       hits=hhsearch_hits)
  return templates_result.features

def set_bfactor(pdb_filename, bfac, idx_res, chains):
  I = open(pdb_filename,"r").readlines()
  O = open(pdb_filename,"w")
  for line in I:
    if line[0:6] == "ATOM  ":
      seq_id = int(line[22:26].strip()) - 1
      seq_id = np.where(idx_res == seq_id)[0][0]
      O.write(f"{line[:21]}{chains[seq_id]}{line[22:60]}{bfac[seq_id]:6.2f}{line[66:]}")
  O.close()

def predict_structure(prefix, feature_dict, Ls, model_params, use_model, do_relax=False, random_seed=0):  
  """Predicts structure using AlphaFold for the given sequence."""

  # Minkyung's code
  # add big enough number to residue index to indicate chain breaks
  idx_res = feature_dict['residue_index']
  L_prev = 0
  # Ls: number of residues in each chain
  for L_i in Ls[:-1]:
      idx_res[L_prev+L_i:] += 200
      L_prev += L_i  
  chains = list("".join([ascii_uppercase[n]*L for n,L in enumerate(Ls)]))
  feature_dict['residue_index'] = idx_res

  # Run the models.
  plddts,paes = [],[]
  unrelaxed_pdb_lines = []
  relaxed_pdb_lines = []

  for model_name, params in model_params.items():
    if model_name in use_model:
      print(f"running {model_name}")
      # swap params to avoid recompiling
      # note: models 1,2 have diff number of params compared to models 3,4,5
      if any(str(m) in model_name for m in [1,2]): model_runner = model_runner_1
      if any(str(m) in model_name for m in [3,4,5]): model_runner = model_runner_3
      model_runner.params = params
      
      processed_feature_dict = model_runner.process_features(feature_dict, random_seed=random_seed)
      prediction_result = model_runner.predict(processed_feature_dict)
      unrelaxed_protein = protein.from_prediction(processed_feature_dict,prediction_result)
      unrelaxed_pdb_lines.append(protein.to_pdb(unrelaxed_protein))
      plddts.append(prediction_result['plddt'])
      paes.append(prediction_result['predicted_aligned_error'])

      if do_relax:
        # Relax the prediction.
        amber_relaxer = relax.AmberRelaxation(max_iterations=0,tolerance=2.39,
                                              stiffness=10.0,exclude_residues=[],
                                              max_outer_iterations=20)      
        relaxed_pdb_str, _, _ = amber_relaxer.process(prot=unrelaxed_protein)
        relaxed_pdb_lines.append(relaxed_pdb_str)

  # rerank models based on predicted lddt
  lddt_rank = np.mean(plddts,-1).argsort()[::-1]
  out = {}
  print("reranking models based on avg. predicted lDDT")
  for n,r in enumerate(lddt_rank):
    print(f"model_{n+1} {np.mean(plddts[r])}")

    unrelaxed_pdb_path = f'{prefix}_unrelaxed_model_{n+1}.pdb'    
    with open(unrelaxed_pdb_path, 'w') as f: f.write(unrelaxed_pdb_lines[r])
    set_bfactor(unrelaxed_pdb_path, plddts[r], idx_res, chains)

    if do_relax:
      relaxed_pdb_path = f'{prefix}_relaxed_model_{n+1}.pdb'
      with open(relaxed_pdb_path, 'w') as f: f.write(relaxed_pdb_lines[r])
      set_bfactor(relaxed_pdb_path, plddts[r], idx_res, chains)

    out[f"model_{n+1}"] = {"plddt":plddts[r], "pae":paes[r]}
  return out

In [ ]:
#@title Call MMseqs2 to get MSA/templates
if use_templates:
  a3m_lines, template_paths = cf.run_mmseqs2(query_sequence, jobname, use_env, use_templates=True)
  if template_paths is None:
    template_features = mk_mock_template(query_sequence * homooligomer)
  else:
    template_features = mk_template(a3m_lines, template_paths)
elif use_msa:
  a3m_lines = cf.run_mmseqs2(query_sequence, jobname, use_env)
  template_features = mk_mock_template(query_sequence * homooligomer)
else:
  template_features = mk_mock_template(query_sequence * homooligomer)

if use_msa:
  with open(a3m_file, "w") as text_file:
    text_file.write(a3m_lines)
else:
  a3m_lines = "".join(open(a3m_file,"r").read())

# parse MSA
msa, deletion_matrix = pipeline.parsers.parse_a3m(a3m_lines)

  0%|          | 0/150 [elapsed: 00:00 remaining: ?]

In [ ]:
#use these lines to install cmake if not installed or older version < 3.15

# Delete the default cmake
#!sudo apt remove cmake
#!sudo apt purge --auto-remove cmake
# Download the required version from the official website, unzip it, and create a build folder -- i.e change the version if needed and copy paste all these:
#!mkdir ~/temp
#%cd ~/temp
#!wget https://cmake.org/files/v3.15/cmake-3.15.7.tar.gz
#!tar -xzvf cmake-3.15.7.tar.gz
#%cd cmake-3.15.7/
# Compile and install
#!./bootstrap
#!make -j4
#!sudo make install
# Movement path
#!sudo cp ./bin/cmake /usr/bin/
# Verify the installation result
#!cmake --version

In [ ]:
#Please install CCMpred and/or cmake if needed on your machine or colab
%cd /content/
!git clone --recursive https://github.com/soedinglab/CCMpred.git
%cd CCMpred
!cmake . -DWITH_CUDA=OFF
!make

In [ ]:
# for colab
%cd /content/

/content


In [ ]:
#Run couplings prediction and feature extraction
! grep -v ">" /content/AMIE_57b5f1.a3m > ccdB.ccm
! /content/CCMpred/bin/ccmpred -r ccdB.raw -n 100 ccdB.ccm ccdB.mat 

 _____ _____ _____               _ 
|     |     |     |___ ___ ___ _| |
|   --|   --| | | | . |  _| -_| . |
|_____|_____|_|_|_|  _|_| |___|___|
                  |_|              

using CPU (1 thread(s))

Reweighted 500 sequences with threshold 0.8 to Beff=427.684 weight mean=0.855368, min=0.0196078, max=1

Will optimize 62686429 32-bit variables

iter	eval	f(x)    	║x║     	║g║     	step
1   	1   	271195  	62063   	9.5812947e+08	2.43e-05
2   	1   	253919  	62063.2 	4.3376979e+08	2.47e-05
3   	1   	237908  	62062.7 	2.2654498e+08	4.54e-05
4   	1   	221317  	62059.4 	1.7439966e+08	8.74e-05
5   	1   	204267  	62054.7 	1.176354e+08	0.00011
6   	1   	188157  	62059.7 	1.0005502e+08	0.000157
7   	1   	172961  	62090.9 	88350424	0.0002
8   	2   	164326  	62118.5 	54592096	0.00013
9   	1   	156326  	62145.9 	45216112	0.00019
10  	1   	149269  	62175.8 	40003620	0.000253
11  	2   	145175  	62196.4 	25951848	0.000175
12  	2   	143210  	62203.6 	18078644	0.000132
13  	1   	141377  	62201.4 	142

In [ ]:
#Detect 2Dmaps
!/content/CCMpred/scripts/top_couplings.py /content/ccdB.mat

#i	j	confidence
32	10	0.3044931888580322
56	15	0.2980586290359497
29	12	0.22244252264499664
52	17	0.20659537613391876
226	41	0.19266174733638763
33	9	0.19102689623832703
291	280	0.18210677802562714
53	32	0.1762913167476654
23	16	0.17370331287384033
29	14	0.16451257467269897
303	290	0.16417406499385834
303	296	0.1621982604265213
291	277	0.15983903408050537
78	71	0.15886887907981873
178	95	0.1580929309129715
238	231	0.1550264060497284
227	41	0.154866024851799
292	276	0.15271830558776855
290	279	0.15191969275474548
295	276	0.1511610448360443
290	276	0.15080028772354126
294	278	0.1505449414253235
292	278	0.14975517988204956
149	141	0.14973077178001404
296	277	0.1490672528743744
291	279	0.14798828959465027
293	279	0.14739006757736206
303	292	0.1464724838733673
296	279	0.14625784754753113
293	277	0.14591403305530548


In [ ]:
#remove hashes
!sed /^#/d /content/ccdB.raw > mat2.txt

In [ ]:
#Determine the last row for singletons for later dataframe creations (The last lline before a hash)
!grep -n -2  '# 0 1'  ccdB.raw

376--6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	
377--6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	-6.21929	
378:# 0 1
379--4.71698876935988664627e-05	-3.64249922313319984823e-06	-9.78515599854290485382e-05	-2.87692273559514433146e-05	0.00000000000000000000e+00	-2.46655381488380953670e-05	-3.22286068694666028023e-05	-4.31307898907107301056e-06	0.00000000000000000000e+00	-5.24184224559576250613e-06	-3.67668576473079156131e-06	-8.88990143721457570791e-06	-4.81935421703383326530e-06	0.00000000000000000000e+00	-4.05736282118596136570e-05	-3.48763249348849058151e-04	-3.37197794578969478607e-04	0.00000000000000000000e+00	0.00000000000000000000e+00	-2.76616060546075459570e-06	-8.24900344014167785645e-04	
380--4.71698876935988664627e-05	

In [ ]:
import pandas as pd
# For skip rows, use the number provided from the last cell
mat2=pd.read_csv('/content/mat2.txt',delimiter="\t", error_bad_lines=False,header=None,skiprows=377 ) 
mat1=pd.read_csv('/content/mat2.txt',delimiter="\t", error_bad_lines=False,header=None,nrows=377 )
mat2=mat2.transpose()
mat1=mat1.transpose()
#mat2 = mat2.iloc[: , :-1]
mat2.drop(mat2.tail(1).index,inplace=True) # drop last row
mat1.drop(mat1.tail(1).index,inplace=True) # drop last row

mat1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376
0,-5.239680,-2.716660,-2.15090,-2.59136,-2.45244,-2.755510,-0.842177,0.481063,0.701072,2.530800,4.217410,1.799110,1.612210,-0.003963,2.940040,1.613960,3.370220,0.693159,2.675190,1.191950,1.620370,3.115490,1.892450,2.948440,2.765780,2.583470,5.971780,2.587780,1.720040,2.218620,2.970170,1.394970,0.694433,3.170130,1.619400,3.681990,4.045370,1.611470,-0.005268,3.110500,...,-2.23917,-3.26880,-3.13285,-3.43602,-2.63582,-4.42247,-3.39732,-4.26654,-3.25278,-3.83298,-4.54122,-3.51918,-4.43311,-3.30383,-4.27792,-4.14376,-4.05766,-5.45789,-4.78557,-4.37118,-5.07442,-6.18354,-6.14795,-4.99495,-5.43051,-6.14976,-6.14543,-6.18295,-6.18295,-6.16199,-6.16199,-6.16199,-6.09655,-6.09655,-6.21929,-6.21929,-6.21929,-6.21929,-6.21929,-6.21929
1,-5.239680,-4.596330,-2.54614,-3.87039,-2.61489,-3.276160,-2.815490,-1.614460,0.000233,3.015100,-0.004399,4.972850,2.094890,1.106060,4.390090,1.102910,3.174820,1.619250,3.663160,0.401425,3.979050,2.847700,3.761940,3.011320,1.625450,4.651730,1.958450,1.106260,4.735730,3.058500,0.694420,1.391100,1.105390,2.203320,-0.004988,3.831830,4.449240,2.658260,2.901970,1.802600,...,-2.73589,-3.60170,-2.82025,-3.34528,-3.38072,-2.78275,-3.32888,-2.58724,-3.00946,-3.33737,-3.84115,-4.39419,-4.23949,-4.10307,-5.38800,-4.72302,-5.03451,-4.75602,-6.18341,-5.48770,-6.17955,-5.07293,-3.19875,-6.10210,-6.13063,-6.14976,-6.14543,-6.18295,-6.18295,-6.16199,-6.16199,-6.16199,-6.09655,-6.09655,-6.21929,-6.21929,-6.21929,-6.21929,-6.21929,-6.21929
2,-5.239680,-1.887020,-2.01461,-2.06342,-2.33514,-0.729550,-2.649690,-0.915167,-0.698270,1.708060,-0.004399,3.367250,4.993580,1.802400,2.498340,1.103610,2.712410,-0.005107,1.952180,0.225458,-0.409075,2.157540,0.922223,1.094760,0.702467,2.720230,1.103140,-0.004367,3.230390,4.106920,0.692555,3.437510,1.105790,2.640860,0.694193,3.498540,-0.006108,2.567910,1.104440,1.098840,...,-4.02146,-1.68018,-4.61687,-4.13740,-5.09119,-4.71667,-5.82979,-5.18404,-4.11989,-4.13596,-5.24358,-5.32695,-4.94741,-4.28741,-6.08785,-6.10811,-6.14124,-5.45807,-6.18341,-6.18750,-6.17955,-6.18354,-6.14795,-6.10210,-6.13063,-6.14976,-6.14543,-6.18295,-6.18295,-6.16199,-6.16199,-6.16199,-6.09655,-6.09655,-6.21929,-6.21929,-6.21929,-6.21929,-6.21929,-6.21929
3,-5.239680,-3.205900,-3.31567,-1.83791,-3.10590,-0.855623,-2.805860,-0.505265,0.000340,2.805960,1.107580,3.641020,5.452740,1.965320,2.586830,1.623460,2.792010,1.103660,3.750220,-0.694479,2.559510,3.783730,1.401270,2.893630,0.703216,1.953620,0.692198,0.695238,4.255460,4.480410,2.412450,3.899840,1.967030,5.098600,2.094860,3.825830,2.767490,1.394430,2.203690,-0.005998,...,-3.53109,-2.13045,-3.32054,-3.94749,-2.91269,-3.72701,-4.43809,-3.55869,-4.09757,-4.81806,-4.83148,-5.32719,-4.95021,-4.67356,-6.08785,-5.00177,-4.73755,-3.77298,-5.48544,-5.48968,-5.07148,-5.07739,-4.52201,-6.10210,-6.13063,-6.14976,-6.14543,-6.18295,-6.18295,-6.16199,-6.16199,-6.16199,-6.09655,-6.09655,-6.21929,-6.21929,-6.21929,-6.21929,-6.21929,-6.21929
4,-5.239680,-5.293120,-5.51869,-5.49358,-5.52237,-5.080430,-3.923220,-1.614460,-0.698270,-0.699078,0.694015,-0.006113,-0.006005,-0.003963,2.195540,-0.005345,1.607320,-0.005107,-0.700127,-1.392170,-1.105200,-1.105320,-0.698447,1.389670,-0.698896,0.690031,-0.005442,0.694541,0.000792,-0.005528,-0.005201,0.691466,-0.004599,-0.006594,-0.004988,1.091360,-0.006108,1.099340,-0.005268,2.400160,...,-4.93723,-5.69451,-5.72106,-5.05846,-5.09058,-5.82090,-5.82979,-5.88129,-5.90583,-5.92525,-5.94354,-6.02459,-6.05584,-6.07369,-6.08785,-6.10811,-6.14124,-6.15610,-6.18341,-6.18750,-6.17955,-6.18354,-5.44973,-6.10210,-6.13063,-6.14976,-6.14543,-6.18295,-6.18295,-6.16199,-6.16199,-6.16199,-6.09655,-6.09655,-6.21929,-6.21929,-6.21929,-6.21929,-6.21929,-6.21929
5,-5.239680,-3.340160,-2.94494,-2.8297

In [ ]:
# functions for encoding local evolutioanry features
def singlocator(x,y):
  if str(x)=='-' :
    return 0
  else:
    pass
  if str(x)=='*' :
    return 0
  else:
    pass
  list1=[]
  list1[:0]=x
  out1=0
  xdict = {'A': 1, 'C': 5, 'E': 7, 'D': 4, 'G': 8, 'F': 14, 'I': 10, 'H': 9, 'K': 12, 'M': 13, 'L': 11,
              'N': 3, 'Q': 6, 'P': 15, 'S': 16, 'R': 2,'T': 17, 'W': 18, 'V': 20, 'Y': 19}
  for i in list1:
    out1=int(xdict[i])
  q=mat1.iat[out1-1,y-1]
  return q
def pairlocator(x,y,z,l,s):
  if y>l:
    r=y
    y=l
    l=r
    w=x
    x=z
    z=w
  elif y==l:
    return 1
  else:
    pass
  list1=[]
  list1[:0]=x
  list2=[]
  list2[:0]=z
  out1=0
  out2=0
  xdict = {'A': 1, 'C': 5, 'E': 7, 'D': 4, 'G': 8, 'F': 14, 'I': 10, 'H': 9, 'K': 12, 'M': 13, 'L': 11,
              'N': 3, 'Q': 6, 'P': 15, 'S': 16, 'R': 2, '-':21,'T': 17, 'W': 18, 'V': 20, 'Y': 19,'*':21}
  for i in list1:
    out1=int(xdict[i])

  for i in list2:
    out2=int(xdict[i])

  y3=(l-y-1)*21
  k=0
  for i in range(len(s)-(y-1), len(s)):   
    k+=i*21
  q=mat2.iat[out2-1,(k)+y3+out1-1]
  return q

def encoder(s):
  q=[]
  sing=[]
  t=[]
  for l in range(0,len(s)):
    sing.append(singlocator(s[l],l+1))
    for i in range(0,len(s)):
      q.append(pairlocator(s[l],l,s[i],i,s))
  t.extend(sing)
  t.extend(q)

  return t
def encoder2(s,x1):
  q=[]
  sing=[]
  t=[]
  for l in range(0,len(s)):
    sing.append(singlocator(s[l],l+1))
    for i in (x1):

      q.append(pairlocator(s[l],l,s[i],i,s))
  t.extend(sing)
  t.extend(q)

  return t

import numpy as np
def createList(r1, r2):
    return np.arange(r1, r2+1, 1)




In [ ]:
# Generate a padded alignment
from Bio import AlignIO
from Bio import SeqIO
from Bio import Seq
import os

input_file1 = '/content/AMIE_57b5f1.a3m'

recordss = SeqIO.parse(input_file1, 'fasta')
records = list(recordss)         
maxlen = max(len(record.seq) for record in records)


# pad sequences so that they all have the same length
for record in records:
    if len(record.seq) != maxlen:
        sequence = str(record.seq).ljust(maxlen, '-')
        record.seq = Seq.Seq(sequence)
assert all(len(record.seq) == maxlen for record in records)

# write to temporary file and do alignment
output_file = '{}_padded.fasta'.format(os.path.splitext(input_file1)[0])
with open(output_file, 'w') as f:
    SeqIO.write(records, f, 'fasta')
alignment = AlignIO.read(output_file, "fasta")
print(alignment)

[SeqRecord(seq=Seq('MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGESGSGKSVL...LAD'), id='101', name='101', description='101', dbxrefs=[]), SeqRecord(seq=Seq('MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGESGSGKSVL...LAD'), id='UniRef100_UPI0005E80636', name='UniRef100_UPI0005E80636', description='UniRef100_UPI0005E80636\t421\t0.994\t5.957E-128\t0\t354\t355\t0\t354\t355', dbxrefs=[]), SeqRecord(seq=Seq('MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGESGSGKSVL...LAD'), id='UniRef100_UPI0007692630', name='UniRef100_UPI0007692630', description='UniRef100_UPI0007692630\t417\t0.994\t7.370E-127\t0\t354\t355\t0\t354\t355', dbxrefs=[]), SeqRecord(seq=Seq('MTKESNVILTARDIVVEFDVRDKVLTAIRGVSLELIEGEVLALVGESGSGKSVL...LED'), id='UniRef100_A0A139RPK9', name='UniRef100_A0A139RPK9', description='UniRef100_A0A139RPK9\t413\t0.935\t3.206E-125\t0\t354\t355\t0\t354\t355', dbxrefs=[]), SeqRecord(seq=Seq('MTKEENVILTARDIVVEFDVRDRVLTAIRGVSLDLIEGEVLALVGESGSGKSVL...L--'), id='UniRef100_UPI001AE2F01C', name='UniRef100_U

In [ ]:
# prepare a clean alignment file for frequency calculations
!cat /content/AMIE_57b5f1_padded.fasta
!awk '{print $0 (/^>/ ? "_" (++c[$1]) : "")}' /content/AMIE_57b5f1_padded.fasta > alignment.fasta
!sed -i 's/.*/\U&/' alignment.fasta
#if you prefer ClustalW
!grep -v ">"  alignment.fasta > aln.aln


>101
MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGESGSGKSVLTKTFTG
MLEENGRIAQGSIDYRGQDLTALSSHKDWEQIRGAKIATIFQDPMTSLDPIKTIGSQITE
VIVKHQGKTAKEAKELAIDYMNKVGIPDADRRFNEYPFQYSGGMRQRIVIAIALACRPDV
LICDEPTTALDVTIQAQIIDLLKSLQNEYHFTTIFITHDLGVVASIADKVAVMYAGEIVE
YGTVEEVFYDPRHPYTWSLLSSLPQLADDKGDLYSIPGTPPSLYTDLKGDAFALRSDYAM
QIDFEQKAPQFSVSETHWAKTWLLHEDAPKVEKPAVIANLHDKIREKMGFAHLAD-----
-----------------
>UniRef100_UPI0005E80636	421	0.994	5.957E-128	0	354	355	0	354	355
MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGESGSGKSVLTKTFTG
MLEENGRIAQGSIDYRGQDLTALSSHKEWEQIRGAKIATIFQDPMTSLDPIKTIGSQITE
VIVKHQGKTAKEAKELAIDYMNKVGIPDADRRFNEYPFQYSGGMRQRIVIAIALACRPDV
LICDEPTTALDVTIQAQIIDLLKSLQNEYHFTTIFITHDLGVVASIADKVAVMYAGEIVE
YGTVEEVFYDPRHPYTWSLLSSLPQLADDKGDLYSIPGTPPSLYTDLKGDAFALRSDYAR
QIDFEQKAPQFSVSETHWAKTWLLHEDAPKVEKPAVIANLHDKIREKMGFAHLAD-----
-----------------
>UniRef100_UPI0007692630	417	0.994	7.370E-127	0	354	355	0	354	355
MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGESGSGKSVLTKTFTG
MLEENGRIAQDSIDYRSQDLTALSSHKDWEQIRG

In [ ]:
#calculate observed frequencies (Might take time, depending on size of the alignment and length of protein)
from Bio import AlignIO
filename = "/content/alignment.fasta"
alignment = AlignIO.read(filename, "fasta")

#The substitutions property of the alignment stores the number of times different residues substitute for each other:
observed_frequencies = alignment.substitutions


         D        E        H        K        R
D 436786.0 203226.5  63391.0  89456.5 142145.5
E 203226.5 674960.0  51331.5 151650.0 202381.0
H  63391.0  51331.5 220417.0  38304.0  65582.0
K  89456.5 151650.0  38304.0 305502.0 188243.5
R 142145.5 202381.0  65582.0 188243.5 543525.0

D 0.2045
E 0.2807
H 0.0960
K 0.1691
R 0.2497

       D      E      H      K      R
D 0.0418 0.0574 0.0196 0.0346 0.0511
E 0.0574 0.0788 0.0270 0.0475 0.0701
H 0.0196 0.0270 0.0092 0.0162 0.0240
K 0.0346 0.0475 0.0162 0.0286 0.0422
R 0.0511 0.0701 0.0240 0.0422 0.0624



In [ ]:
#Calculate evolutioanry distances (might take time depending on size of the alignment)
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio import AlignIO
import numpy as np
aln2 = AlignIO.read(open('/content/alignment.fasta'), 'fasta')

calculator = DistanceCalculator('blosum62')
dm = calculator.get_distance(aln2)
import pandas as pd
matm=pd.read_csv('/content/ccdB.mat', delimiter="	", header=None)
matm=matm.astype(float)
matm['magn']=0
for i in range(0,len(matm.index)):
  x=np.array((matm.loc[i].values.tolist()))
  x1=np.array(x[:-2])
  y=np.sqrt(x1.dot(x1))
  matm['magn'].loc[i]=y
matm["mean"]=matm.mean(axis=0)



Streaming output truncated to the last 5000 lines.
 0.04097756 0.01984973 0.05194189 0.03987688 0.0248569  0.04163609
 0.04742405 0.0235005  0.03893508 0.03796729 0.02875158 0.04369567
 0.03024465 0.02221777 0.02562914 0.02757899 0.05658471 0.02157085
 0.02554581 0.02872998 0.04163541 0.03975031 0.03332477 0.03105631
 0.04687593 0.037517   0.02759589 0.04083186 0.02934007 0.03794789
 0.04830013 0.04544623 0.06718314 0.08522834 0.10703179 0.06906746
 0.09529467 0.13166466 0.11387572 0.11949971 0.10304465 0.09442363
 0.09596957 0.08922528 0.07161118 0.0836954  0.0791339  0.05394754
 0.09897589 0.07553051 0.11582048 0.15932499 0.09972884 0.13517483
 0.09967456 0.12283192 0.12741114 0.         0.10686687 0.08166991
 0.07874262 0.09744816 0.07949054 0.0768702  0.12279497 0.06215239
 0.07155126 0.05759147 0.05857848 0.0715677  0.0655469  0.07832707
 0.06899261 0.06590432 0.07807375 0.07476915 0.05003239 0.05554192
 0.06565547 0.05914432 0.05423936 0.08051571 0.11905247 0.07304803
 0.08680035

In [ ]:
#compute distances with query sequence, used for one of the methods for epistatic effects calculations
def epidmcalc2( string_to_search1,x1):
  import math
  xy=dict()
  xyz=[]
  res = []
  xyy=[]

  for (i) in x1:
    q=(matm["magn"][int(i-1)])
    string_to_search=string_to_search1[int(i)-1:int(i)]
    line_number = 0
    list_of_results = []
    
    a=[]
    
    with open('/content/ccdB.ccm', 'r') as read_obj:
      # Read all lines in the file one by one
      for line in read_obj:
  
        line_number += 1

        if string_to_search in str(line)[i-1:i]:
          list_of_results.append((line_number, line.rstrip()))      
 

          for elem in list_of_results: 
            x=elem[0]-1
            xx=(dm[x][1])
            xy[i]=xx
        
          for l in xy:

            xyy.append(np.min(xy[i]*q))
            [res.append(x) for x in xyy if x not in res]
            
            
      else:
        res.append(q*0.99)

  return (sum(res))


In [ ]:
#Use this function for calulating indepdent mutational effects 
import math
from Bio.Align import AlignInfo
import re
summary_align = AlignInfo.SummaryInfo(alignment)
consensus1 = summary_align.dumb_consensus()
my_pssm = summary_align.pos_specific_score_matrix(consensus1)
def indp2(x1):
  
  a=[]

  if str(x1) != '' :
    
    x=x1.split(", ")
    for i in x:
      if '*' in i:
        pass
      else:
        z=re.sub(r'[A-Z]+', '', str(i), re.I) 
        ref = len(summary_align.get_column(int(z)))

        s1=i.split(str(z))
        if str((s1[1]))=="":
          pass
        else:
          s=str((s1[0]))+str((s1[1]))
          observed_frequencies1 = observed_frequencies.select(s)
         
          fincons=observed_frequencies1[1][0]
        
          z=int(z)
          poscons=sum(my_pssm[int(z-1)].values())/ref
         
          logcons=((my_pssm[int(z-1)][s1[1]])-math.log(fincons+0.000001))*poscons
        
          a.append(logcons)
  elif str(x1) == '' :
    a=[0]
  return sum(a)

-4.495858008828584


In [ ]:
# use this function as a method for computing epistatic effects
import math
from Bio.Align import AlignInfo
import re
summary_align = AlignInfo.SummaryInfo(alignment)
consensus1 = summary_align.dumb_consensus()
my_pssm = summary_align.pos_specific_score_matrix(consensus1)
def epis3(x1):
  
  a=[]
  if str(x1) != '' :
    x=x1.split(", ")
    for i in x:
      if '*' in i:
        pass
      else:
        z=re.sub(r'[A-Z]+', '', str(i), re.I) 
        ref = len(summary_align.get_column(int(z)))
        s1=i.split(str(z))
        if str((s1[1]))=="":
          pass
        else:
          s=str((s1[0]))+str((s1[1]))
          observed_frequencies1 = observed_frequencies.select(s)
          z=int(z)
          logcons=((math.log((my_pssm[int(z-1)][s1[1]])/(my_pssm[int(z-1)][s1[0]])+0.0000001)))
          a.append(logcons)
  elif str(x1) == '' :
    a=[0]
  return sum(a)


1.3862943861198904


In [ ]:
#use to encode Georgiev 19 PCA features for physicochemical parameters (feel free to adjust the code for a more neat way)
def Greg1(s):
  import numpy as np
  A=[0.57,3.37,-3.66,2.34,-1.07,-0.40,1.23,-2.32,-2.01,1.31,-1.14,0.19,1.66,4.39,0.18,-2.60,1.49,0.46,-4.22]
  R=[-2.80,0.31,2.84,0.25,0.20,-0.37,3.81,0.98,2.43,-0.99,-4.90,2.09,-3.08,0.82,1.32,0.69,-2.62,-1.49,-2.57]
  N=[-2.02,-1.92,0.04,-0.65,1.61,2.08,0.40,-2.47,-0.07,7.02,1.32,-2.44,0.37,-0.89,3.13,0.79,-1.54,-1.71,-0.25]
  D=[-2.46,-0.66,-0.57,0.14,0.75,0.24,-5.15,-1.17,0.73,1.50,1.51,5.61,-3.85,1.28,-1.98,0.05,0.90,1.38,-0.03]
  C=[2.66,-1.52,-3.29,-3.77,2.96,-2.23,0.44,-3.49,2.22,-3.78,1.98,-0.43,1.03,0.93,1.43,1.45,-1.15,-1.64,-1.05]
  Q=[-2.54,1.82,-0.82,-1.85,0.09,-0.60,0.25,2.11,-1.92,-1.67,0.70,-0.27,-0.99,-1.56,6.22,-0.18,2.72,4.35,0.92]
  E=[-3.08,3.45,0.05,0.62,0.49,0.00,5.66,0.11,1.49,2.26,1.62,3.97,2.30,0.06,-0.35,1.51,-2.29,-1.47,0.15]
  G=[0.15,-3.49,-2.97,2.06,0.70,7.47,0.41,1.62,-0.47,-2.90,-0.98,-0.62,-0.11,0.15,-0.53,0.35,0.30,0.32,0.05]
  H=[-0.39,1.00,-0.63,-3.49,0.05,0.41,1.61,-0.60,3.55,1.52,-2.28,-3.12,-1.45,-0.77,-4.18,-2.91,3.37,1.87,2.17]
  I=[3.10,0.37,0.26,1.04,-0.05,-1.18,-0.21,3.45,0.86,1.98,0.89,-1.67,-1.02,-1.21,-1.78,5.71,1.54,2.11,-4.18]
  L=[2.72,1.88,1.92,5.33,0.08,0.09,0.27,-4.06,0.43,-1.20,0.67,-0.29,-2.47,-4.79,0.80,-1.43,0.63,-0.24,1.01]
  K=[-3.89,1.47,1.95,1.17,0.53,0.10,4.01,-0.01,-0.26,-1.66,5.86,-0.06,1.38,1.78,-2.71,1.62,0.96,-1.09,1.36]
  M=[1.89,3.88,-1.57,-3.58,-2.55,2.07,0.84,1.85,-2.05,0.78,1.53,2.44,-0.26,-3.09,-1.39,-1.02,-4.32,-1.34,0.09]
  F=[3.12,0.68,2.40,-0.35,-0.88,1.62,-0.15,-0.41,4.20,0.73,-0.56,3.54,5.25,1.73,2.14,1.10,0.68,1.46,2.33]
  P=[-0.58,-4.33,-0.02,-0.21,-8.31,-1.82,0.12,-1.18,0.00,-0.66,0.64,-0.92,-0.37,0.17,0.36,0.08,0.16,-0.34,0.04]
  S=[-1.10,-2.05,-2.19,1.36,1.78,-3.36,1.39,-1.21,-2.83,0.39,-2.92,1.27,2.86,-1.88,-2.42,1.75,-2.77,3.36,2.67]
  T=[-0.65,-1.60,-1.39,0.63,1.35,-2.45,-0.65,3.43,0.34,0.24,-0.53,1.91,2.66,-3.07,0.20,-2.20,3.73,-5.46,-0.73]
  W=[1.89,-0.09,4.21,-2.77,0.72,0.86,-1.07,-1.66,-5.87,-0.66,-2.49,-0.30,-0.50,1.64,-0.72,1.75,2.73,-2.20,0.90]
  Y=[0.79,-2.62,4.11,-0.63,1.89,-0.53,-1.30,1.31,-0.56,-0.95,1.91,-1.26,1.57,0.20,-0.76,-5.19,-2.56,2.87,-3.43]
  V=[2.64,0.03,-0.67,2.34,0.64,-2.01,-0.33,3.93,-0.21,1.27,0.43,-1.71,-2.93,4.22,1.06,-1.31,-1.97,-1.21,4.77]
  
  list1=[]
  list1[:0]=s
  out=[]
  out1=0
  b_factor1 = {'A': A[0],'C': C[0], 'E': E[0], 'D': D[0], 'G':G[0], 'F': F[0], 'I': I[0], 'H': H[0],
              'K': K[0],
              'M': M[0], 'L': L[0],'*':'0.0','N':N[0], 'Q': Q[0], 'P': P[0], 'S':S[0], 'R': R[0], 'T': T[0], 'W': W[0], 'V': V[0],'*':'0.0', 'Y': Y[0]}
  for i in list1:
    out1+=float(b_factor1[i])
  out.extend([out1/len(s)])
  out2=0
  list2=[]
  list2[:0]=s
  b_factor2 = {'A': A[1],'C': C[1], 'E': E[1], 'D': D[1], 'G':G[1], 'F': F[1], 'I': I[1], 'H': H[1],
              'K': K[1],
              'M': M[1], 'L': L[1],'*':'0.0','N':N[1], 'Q': Q[1], 'P': P[1], 'S':S[1], 'R': R[1], 'T': T[1], 'W': W[1], 'V': V[1],'*':'0.0', 'Y': Y[1]}
  for i in list2:
    out2+=float(b_factor2[i])
  out.extend([out2/len(s)])
  out3=0
  list3=[]
  list3[:0]=s
  b_factor3 = {'A': A[2],'C': C[2], 'E': E[2], 'D': D[2], 'G':G[2], 'F': F[2], 'I': I[2], 'H': H[2],
              'K': K[2],
              'M': M[2], 'L': L[2],'*':'0.0','N':N[2], 'Q': Q[2], 'P': P[2], 'S':S[2], 'R': R[2], 'T': T[2], 'W': W[2], 'V': V[2],'*':'0.0', 'Y': Y[2]}
  for i in list3:
    out3+=float(b_factor3[i])
  out.extend([out3/len(s)])
  out4=0
  list4=[]
  list4[:0]=s
  b_factor4 = {'A': A[3],'C': C[3], 'E': E[3], 'D': D[2], 'G':G[3], 'F': F[3], 'I': I[3], 'H': H[3],
              'K': K[3],
              'M': M[3], 'L': L[3],'*':'0.0','N':N[3], 'Q': Q[3], 'P': P[3], 'S':S[3], 'R': R[3], 'T': T[3], 'W': W[3], 'V': V[3],'*':'0.0', 'Y': Y[3]}
  for i in list4:
    out4+=float(b_factor4[i])
  out.extend([out4/len(s)])
  out5=0
  list5=[]
  list5[:0]=s
  b_factor5 = {'A': A[4],'C': C[4], 'E': E[4], 'D': D[4], 'G':G[4], 'F': F[4], 'I': I[4], 'H': H[4],
              'K': K[4],
              'M': M[4], 'L': L[4],'*':'0.0','N':N[4], 'Q': Q[4], 'P': P[4], 'S':S[4], 'R': R[4], 'T': T[4], 'W': W[4], 'V': V[4],'*':'0.0', 'Y': Y[4]}
  for i in list5:
    out5+=float(b_factor5[i])
  out.extend([out5/len(s)])
  out6=0
  list6=[]
  list6[:0]=s
  b_factor6 = {'A': A[5],'C': C[5], 'E': E[5], 'D': D[5], 'G':G[5], 'F': F[5], 'I': I[5], 'H': H[5],
              'K': K[5],
              'M': M[5], 'L': L[5],'*':'0.0','N':N[5], 'Q': Q[5], 'P': P[5], 'S':S[5], 'R': R[5], 'T': T[5], 'W': W[5], 'V': V[5],'*':'0.0', 'Y': Y[5]}
  for i in list6:
    out6+=float(b_factor6[i])
  out.extend([out6/len(s)])
  out7=0
  list7=[]
  list7[:0]=s
  b_factor7 = {'A': A[6],'C': C[6], 'E': E[6], 'D': D[6], 'G':G[6], 'F': F[6], 'I': I[6], 'H': H[6],
              'K': K[6],
              'M': M[6], 'L': L[6],'*':'0.0','N':N[6], 'Q': Q[6], 'P': P[6], 'S':S[6], 'R': R[6], 'T': T[6], 'W': W[6], 'V': V[6],'*':'0.0', 'Y': Y[6]}
  for i in list7:
    out7+=float(b_factor7[i])
  out.extend([out7/len(s)])
  out8=0
  list8=[]
  list8[:0]=s
  b_factor8 = {'A': A[7],'C': C[7], 'E': E[7], 'D': D[7], 'G':G[7], 'F': F[7], 'I': I[7], 'H': H[7],
              'K': K[7],
              'M': M[7], 'L': L[7],'*':'0.0','N':N[7], 'Q': Q[7], 'P': P[7], 'S':S[7], 'R': R[7], 'T': T[7], 'W': W[7], 'V': V[7],'*':'0.0', 'Y': Y[7]}
  for i in list8:
    out8+=float(b_factor8[i])
  out.extend([out8/len(s)])
  out8=0
  list8=[]
  list8[:0]=s
  out9=0
  list9=[]
  list9[:0]=s
  b_factor9 = {'A': A[8],'C': C[8], 'E': E[8], 'D': D[8], 'G':G[8], 'F': F[8], 'I': I[8], 'H': H[8],
              'K': K[8],
              'M': M[8], 'L': L[8],'*':'0.0','N':N[8], 'Q': Q[8], 'P': P[8], 'S':S[8], 'R': R[8], 'T': T[8], 'W': W[8], 'V': V[8],'*':'0.0', 'Y': Y[8]}
  for i in list9:
    out9+=float(b_factor9[i])
  out.extend([out9/len(s)])
  out10=0
  list10=[]
  list10[:0]=s
  b_factor10 = {'A': A[9],'C': C[9], 'E': E[9], 'D': D[9], 'G':G[9], 'F': F[9], 'I': I[9], 'H': H[9],
              'K': K[9],
              'M': M[9], 'L': L[9],'*':'0.0','N':N[9], 'Q': Q[9], 'P': P[9], 'S':S[9], 'R': R[9], 'T': T[9], 'W': W[9], 'V': V[9],'*':'0.0', 'Y': Y[9]}
  for i in list10:
    out10+=float(b_factor10[i])
  out.extend([out10/len(s)])
  out11=0
  list11=[]
  list11[:0]=s
  b_factor11 = {'A': A[10],'C': C[10], 'E': E[10], 'D': D[10], 'G':G[10], 'F': F[10], 'I': I[10], 'H': H[10],
              'K': K[10],
              'M': M[10], 'L': L[10],'*':'0.0','N':N[10], 'Q': Q[10], 'P': P[10], 'S':S[10], 'R': R[10], 'T': T[10], 'W': W[10], 'V': V[10],'*':'0.0', 'Y': Y[10]}
  for i in list11:
    out11+=float(b_factor11[i])
  out.extend([out11/len(s)])
  out12=0
  list12=[]
  list12[:0]=s
  b_factor12 = {'A': A[11],'C': C[11], 'E': E[11], 'D': D[11], 'G':G[11], 'F': F[11], 'I': I[11], 'H': H[11],
              'K': K[11],
              'M': M[11], 'L': L[11],'*':'0.0','N':N[11], 'Q': Q[11], 'P': P[11], 'S':S[11], 'R': R[11], 'T': T[11], 'W': W[11], 'V': V[11],'*':'0.0', 'Y': Y[11]}
  for i in list12:
    out12+=float(b_factor12[i])
  out.extend([out12/len(s)])
  out13=0
  list13=[]
  list13[:0]=s
  b_factor13 = {'A': A[12],'C': C[12], 'E': E[12], 'D': D[12], 'G':G[12], 'F': F[12], 'I': I[12], 'H': H[12],
              'K': K[12],
              'M': M[12], 'L': L[12],'*':'0.0','N':N[12], 'Q': Q[12], 'P': P[12], 'S':S[12], 'R': R[12], 'T': T[12], 'W': W[12], 'V': V[12],'*':'0.0', 'Y': Y[12]}
  for i in list13:
    out13+=float(b_factor13[i])
  out.extend([out13/len(s)])
  out14=0
  list14=[]
  list14[:0]=s
  b_factor14 = {'A': A[13],'C': C[13], 'E': E[13], 'D': D[13], 'G':G[13], 'F': F[13], 'I': I[13], 'H': H[13],
              'K': K[13],
              'M': M[13], 'L': L[13],'*':'0.0','N':N[13], 'Q': Q[13], 'P': P[13], 'S':S[13], 'R': R[13], 'T': T[13], 'W': W[13], 'V': V[13],'*':'0.0', 'Y': Y[13]}
  for i in list14:
    out14+=float(b_factor14[i])
  out.extend([out14/len(s)])
  out15=0
  list15=[]
  list15[:0]=s
  b_factor15 = {'A': A[14],'C': C[14], 'E': E[14], 'D': D[14], 'G':G[14], 'F': F[14], 'I': I[14], 'H': H[14],
              'K': K[14],
              'M': M[14], 'L': L[14],'*':'0.0','N':N[14], 'Q': Q[14], 'P': P[14], 'S':S[14], 'R': R[14], 'T': T[14], 'W': W[14], 'V': V[14],'*':'0.0', 'Y': Y[14]}
  for i in list15:
    out15+=float(b_factor15[i])
  out.extend([out15/len(s)])
  out16=0
  list16=[]
  list16[:0]=s
  b_factor16 = {'A': A[15],'C': C[15], 'E': E[15], 'D': D[15], 'G':G[15], 'F': F[15], 'I': I[15], 'H': H[15],
              'K': K[15],
              'M': M[15], 'L': L[15],'*':'0.0','N':N[15], 'Q': Q[15], 'P': P[15], 'S':S[15], 'R': R[15], 'T': T[15], 'W': W[15], 'V': V[15],'*':'0.0', 'Y': Y[15]}
  for i in list16:
    out16+=float(b_factor16[i])
  out.extend([out16/len(s)])
  out17=0
  list17=[]
  list17[:0]=s
  b_factor17 = {'A': A[16],'C': C[16], 'E': E[16], 'D': D[16], 'G':G[16], 'F': F[16], 'I': I[16], 'H': H[16],
              'K': K[16],
              'M': M[16], 'L': L[16],'*':'0.0','N':N[16], 'Q': Q[16], 'P': P[16], 'S':S[16], 'R': R[16], 'T': T[16], 'W': W[16], 'V': V[16],'*':'0.0', 'Y': Y[16]}
  for i in list17:
    out17+=float(b_factor17[i])
  out.extend([out17/len(s)])
  out18=0
  list18=[]
  list18[:0]=s
  b_factor18 = {'A': A[17],'C': C[17], 'E': E[17], 'D': D[17], 'G':G[17], 'F': F[17], 'I': I[17], 'H': H[17],
              'K': K[17],
              'M': M[17], 'L': L[17],'*':'0.0','N':N[17], 'Q': Q[17], 'P': P[17], 'S':S[17], 'R': R[17], 'T': T[17], 'W': W[17], 'V': V[17],'*':'0.0', 'Y': Y[17]}
  for i in list18:
    out18+=float(b_factor18[i])
  out.extend([out18/len(s)])
  out19=0
  list19=[]
  list19[:0]=s
  b_factor19 = {'A': A[18],'C': C[18], 'E': E[18], 'D': D[18], 'G':G[18], 'F': F[18], 'I': I[18], 'H': H[18],
              'K': K[18],
              'M': M[18], 'L': L[18],'*':'0.0','N':N[18], 'Q': Q[18], 'P': P[18], 'S':S[18], 'R': R[18], 'T': T[18], 'W': W[18], 'V': V[18],'*':'0.0', 'Y': Y[18]}
  for i in list19:
    out19+=float(b_factor19[i])
  out.extend([out19/len(s)])
  return out
print(Greg1("AMCKL"))

[0.79, 1.816, -0.9299999999999999, 0.29800000000000004, -0.010000000000000023, -0.07400000000000002, 1.3579999999999999, -1.6059999999999999, -0.33399999999999996, -0.9099999999999999, 1.78, 0.36999999999999994, 0.2679999999999999, -0.15600000000000006, -0.338, -0.39599999999999996, -0.47800000000000004, -0.7700000000000001, -0.5619999999999999]


In [ ]:
#functions to be used for physicochemical parameters of mutants (not delta of the paramters between mutant and query)
import Bio
from Bio.SeqUtils.ProtParam import ProteinAnalysis
#Molecular weight function
def physicoMW(s):
  if len(s)>0:
    s=s.replace("*","")
    analysed_seq = ProteinAnalysis(s)
    MW=analysed_seq.molecular_weight()
  else:
    MW=0
  return MW
#Physical instability function
def physicoIns(s):
  if len(s)>0:
    s=s.replace("*","")
    analysed_seq = ProteinAnalysis(s)
    Ins=analysed_seq.instability_index()
  else:
    Ins=0
  return Ins
#Charge at physiolocial Ph function
def physicoPh(s):
  if len(s) >0:
    s=s.replace("*","")
    analysed_seq = ProteinAnalysis(s)
    Ph=analysed_seq.charge_at_pH(7.2)
  else:
    Ph=0
  return Ph
#hydrophobicity Calculations
def physicoGr(s):
  if len(s) >0:
    s=s.replace("*","")
    analysed_seq = ProteinAnalysis(s)
    Gr=analysed_seq.gravy()
  else: 
    Gr=0
  if len(s)==0:
    return 0
  return Gr
#https://www.genome.jp/entry/aaindex:HUTJ700102
#I    A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#   30.88   68.43   41.70   40.66   53.83   46.62   44.98   24.74   65.99   49.71
#   50.62   63.21   55.32   51.06   39.21   35.65   36.50   60.00   51.15   42.75
#//
#Entropy Calculations
def entropycalc(s):
  list1=[]
  list1[:0]=s
  out=0
  out1=0
  b_factor = {'A': '30.88', 'C': '53.83', 'E': '44.98', 'D': '40.66', 'G': '24.74', 'F': '51.06', 'I': '49.71', 'H': '65.99',
              'K': '63.21',
              'M': '55.32', 'L': '50.62','*':'0.0','N': '41.70', 'Q': '46.62', 'P': '53.83', 'S': '46.62', 'R': '68.43', 'T': '36.50', 'W': '60.00', 'V': '42.75','*':'0.0', 'Y': '51.15'}
  for i in list1:
    out1+=float(b_factor[i])
  if len(s)==0:
    return 0
  return out1/len(s)

#Stability Calculations
#https://www.genome.jp/entry/aaindex:TAKK010101
#   A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#     9.8     7.3     3.6     4.9     3.0     2.4     4.4       0    11.9    17.2
#    17.0    10.5    11.9    23.0    15.0     2.6     6.9    24.2    17.2    15.3
#//
def stabcalc(s):

  list1=[]
  list1[:0]=s
  out=0
  out1=0
  b_factor = {'A': '9.8', 'C': '3.0', 'E': '4.4', 'D': '4.9', 'G': '0.0', 'F': '23.0', 'I': '17.2', 'H': '11.9', 'K': '10.5',
              'M': '11.9', 'L': '17.0', 'N': '3.6', '*':'0.0','Q': '2.4', 'P': '15.0', 'S': '2.6', 'R': '7.3', 'T': '6.9', 'W': '24.2', 'V': '15.3', 'Y': '17.2'}
  for i in list1:
    out1+=float(b_factor[i])
  if len(s)==0:
    return 0
  return out1/len(s)
#https://www.genome.jp/entry/aaindex:HUTJ700101
#I    A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#   29.22   26.37   38.30   37.09   50.70   44.02   41.84   23.71   59.64   45.00
#   48.03   57.10   69.32   48.52   36.13   32.40   35.20   56.92   51.73   40.35
#//
#Thermal Capacity Calculations
def thermalcapcalc(s):
  list1=[]
  list1[:0]=s
  out1=0
  b_factor = {'A': '29.22', 'C': '50.70', 'E': '41.84', 'D': '37.09', 'G': '23.71', 'F': '48.52', 'I': '45.00', 'H': '59.64', 'K': '57.10',
              'M': '69.32', 'L': '48.03', 'N': '38.30', '*':'0.0','Q': '44.02', 'P': '36.13', 'S': '32.40', 'R': '26.37', 'T': '35.20', 'W': '56.92', 'V': '40.35', 'Y': '51.73'}

  for i in list1:
    out1+=float(b_factor[i])
  if len(s)==0:
    return 0
  return out1/len(s)
#https://www.genome.jp/dbget-bin/www_bget?aaindex:BIGC670101
# A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#    52.6   109.1    75.7    68.4    68.3    89.7    84.7    36.3    91.9   102.0
#   102.0   105.1    97.7   113.9    73.6    54.9    71.2   135.4   116.2    85.1
#Protein Volume Calculations
def Volcalc(s):
  list1=[]
  list1[:0]=s
  out=0
  out1=0
  b_factor = {'A': '52.6', 'C': '68.3', 'E': '84.7', 'D': '68.4', 'G': '36.3','*':'0.0', 'F': '113.9', 'I': '102.0', 'H': '91.9', 'K': '105.1', 'M': '97.7', 'L': '102.0', 'N': '75.7', 'Q': '89.7', 'P': '73.6', 'S': '54.9', 'R': '109.1', 'T': '71.2', 'W': '36.3', 'V': '85.1', 'Y': '116.2'}
  for i in list1:
    out1+=float(b_factor[i])
  if len(s)==0:
    return 0
  return out1/len(s)

#Surface Area
#https://www.genome.jp/entry/aaindex:JANJ780101
#I    A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#    27.8    94.7    60.1    60.6    15.5    68.7    68.2    24.5    50.7    22.8
#    27.6   103.0    33.5    25.5    51.5    42.0    45.0    34.7    55.2    23.7
#//

def SAalc(s):
  list1=[]
  list1[:0]=s
  out1=0

  b_factor = {'A': '27.8', 'C': '15.5', 'E': '68.2', 'D': '60.6', 'G': '24.5', 'F': '25.5', 'I': '22.8', 'H': '50.7', 'K': '103.0', 'M': '33.5', 'L': '27.6',
              'N': '60.1', 'Q': '42.0', 'P': '51.5', 'S': '42.0', 'R': '94.7', '*':'0.0','T': '45.0', 'W': '34.7', 'V': '23.7', 'Y': '55.2'}
  for i in list1:
    out1+=float(b_factor[i])
  if len(s)==0:
    return 0
  return out1/len(s)

#Flexibility
#https://www.genome.jp/dbget-bin/www_bget?aaindex:VINM940101
#I    A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#  0.984   1.008   1.048   1.068   0.906   1.037   1.094   1.031   0.950   0.927
#  0.935   1.102   0.952   0.915   1.049   1.046   0.997   0.904   0.929   0.931
#
#
def Flexcalc(s):
  list1=[]
  list1[:0]=s
  out1=0
  b_factor = {'A': '0.984', 'C': '0.906', 'E': '1.094', 'D': '1.068', 'G': '1.031', 'F': '0.915', 'I': '0.927', 'H': '0.950', 'K': '1.102', 'M': '0.952', 'L': '0.935', 
              'N': '1.048', 'Q': '1.037', 'P': '1.049', 'S': '1.046','*':'0.0', 'R': '1.008', 'T': '0.997', 'W': '0.904', 'V': '0.931', 'Y': '0.929'}
  for i in list1:
    out1+=float(b_factor[i])
  if len(s)==0:
    return 0
  return out1/len(s)



#Previous versions for estimating independet and epistaitc effects of mutations based on frequencies of residues.
#feel free to use
observed_frequencies2 = observed_frequencies.select("ARNDCQEGHILKMFPSTWYVX*")
def EvolFit(s2,s0):
  if len(s2)>0:
    try:
      matrix = substitution_matrices.load("BLOSUM62")
      aligner.substitution_matrix = matrix
      score = aligner.score(s2, consensus)
    except:
     score=0
  else:
     score=0
  return score

def EvolFit2(s2,s0):
  from Bio.Align import substitution_matrices
  aligner1 = Align.PairwiseAligner()
  matrix1 =  matrix1 =observed_frequencies2
  if len(s2)>0:
    try:

      aligner1.substitution_matrix = matrix1
      score = int(aligner1.score(s2, s0))-int(aligner1.score(s0, s0))
    except:
      score=0
  else:
    score=0
  return score   
from Bio import SeqIO, Entrez
from Bio.Seq import Seq


def EvolFit3(s2,s0):
  
  from Bio.Align import substitution_matrices
  aligner1 = Align.PairwiseAligner()
  matrix1 =observed_frequencies2
  aligner1.substitution_matrix = matrix1
  seq1 = Seq(s2)
  seq2 = Seq(consensus)
  icont=int(aligner1.score(seq1, seq2))-int(aligner1.score(seq2, seq2))
  score = aligner1.score(seq1, seq2)-icont
  return score




65.4
0.984


In [ ]:
#supplemntary functions to adjust the final dataframe
def funxii(str1,str2):
  xn1=[]
  s=', '
  for n in range(0,int(len(str1))):
    
      for n in range (0, len(str1)):
        if str1[n] != str2[n]:
          xn1.append(str1[n]+str(n+1)+str2[n])
      xn=s.join(xn1)
      return  xn
def funxiii(str1,str2):
  xn1=[]
  s=', '
  for n in range(0,int(len(str1))):
    
      for n in range (0, len(str1)):
        if str1[n] != str2[n]:
          xn1.append(str2[n])
      xn=s.join(xn1)
      return  xn

def funxi(str1,str2):

  for n in range(0,int(len(str1))):
     
      if str1[n] != str2[n]: return  int(int(n+1))
  
def funx(str1,str2):
  x=[]
  for n in range(0,int(len(str1))):
      
      if str1[n] != str2[n]: 
       
        return str(str1[n] + str(int(n+1)) + str(str2[n]))
def funxw(str1,str2):
  for n in range(0,int(len(str1))):
      if str1[n] != str2[n]: return (str(str2[n]))

def funxs(str1,str2):
  for n in range(0,int(len(str1))):
      if str1[n] != str2[n]: return (str1[n])
def xstop(s):
    s6=''
    
    if "*" in s:
      i=s.find("*")
      s6=s[:i]
    else:
      s6=s
    return s6

def gets(s,x3):
  x4=[]
  x3=[x-1for x in x3]
  for i in x3:
    x4.append(s[i])
  s=''
  l=s.join(x4)
  return l

def substitute2(s, n):
    idxs = random.sample(range(len(s)), n)
    chars = list(s)
    for i in idxs:
        new_base = random.choice(list(proteintein.difference(chars[i])))
        chars[i] = new_base
    return "".join(chars)

def mutate2(s1, d,x1,x2):
  
    s=s1[x1-1:x2+1]
    mutants = set([s])
    while len(mutants) <= (int(factt(len(s),d))*20**d):
        k = d
        for _ in range(k):
            mutant_type = random.choice(["s"])
            if mutant_type == "s":
                mutants.add(substitute2(s,k))
    return list(list(set(mutants)))

def substitute2p(s, n):
    idxs = random.sample(range(len(s)), n)
    chars = list(s)
    for i in idxs:
        new_base = random.choice(list(protein.difference(chars[i])))
        chars[i] = new_base
    return "".join(chars)

def factt(s,d):
  x=[]
  if int(s) >= d:
     x=((math.factorial(int(s))/(math.factorial(d)*(math.factorial(int(s)-d)))))
  else:
    x=d
  return (x)
def factt2(s,d):
  x=[]
  if int(s) >= d:
    for i in range(1,d+1):
      x.append((math.factorial(int(s))/(math.factorial(i)*(math.factorial(int(s)-i))))*20**i)
  else:
    x=d
  return sum(x)
def mutate2p(s, d):
    mutants = set([s])
    while len(mutants) < (int(factt(len(s),d))*20**d):
        k = d
        for _ in range(k):
            mutant_type = random.choice(["s"])
            if mutant_type == "s":
                mutants.add(substitute(s,k))
  
    return list(list(set(mutants)))
def substitute3(s, n):
    idxs = n
    chars = list(s)
    for i in idxs:      
        new_base = (list(protein.difference(chars[i])))
        chars[i] = random.choice(new_base)
    return "".join(chars)
def substitute4(s, n):
    idxs = n
    chars = list(s)
    for i in chars:
        new_base = (list(protein.difference(chars[i])))
        chars[i] = new_base
    return "".join(chars)
import random    
def mutate3(s1,x3):
  x3=[x-1 for x in x3]
  mutants = set([s1])
  for i in x3:
    while len(mutants) <= (int(factt(len(x3),len(x3)))*20**len(x3)):
      for _ in range(i):
        mutant_type = random.choice(["s"])
        if mutant_type == "s":
          mutants.add(substitute3(s1,x3))
  
  return (list(set(mutants)))
def seqq(s2,s0,x1,x2):
  if len(s0)==len(s2):
    q=s2
  elif len(s2) <1:
    q="*"
  
  elif x1 <= 1 and x2>1 and  len(s0)>len(s2) :
    q= s2 and s0[:x2+1]
  else:
    q=s0[:x1-1] + s2 + s0[x2:]

  return str(q)

def createList(r1, r2):
    return np.arange(r1, r2+1, 1)


In [ ]:
#physicochemical paramters in seprate functions to calculate delta of change from query sequence
from Bio.SeqUtils.ProtParam import ProteinAnalysis
#Delta of Molecular Weight
def dphysicoMW(s,s2):
  if len(s)>0 and len(s2)>0:
    s=s.replace("*","")
    analysed_seq = ProteinAnalysis(s)
    analysed_seq2 = ProteinAnalysis(s2)
    MW=analysed_seq2.molecular_weight()-analysed_seq.molecular_weight()
  else:
    MW=-(ProteinAnalysis(s).molecular_weight())
  return MW
#Delta of Physical Instability
def dphysicoIns(s,s2):
  if len(s)>0 and len(s2)>0:
    s=s.replace("*","")
    analysed_seq = ProteinAnalysis(s)
    analysed_seq2 = ProteinAnalysis(s2)
    Ins=analysed_seq2.instability_index()-analysed_seq.instability_index()
  else:
    Ins=-(ProteinAnalysis(s).instability_index())
  return Ins
#delta of Ph change at physiological Ph
def dphysicoPh(s,s2):
  if len(s)>0 and len(s2)>0:
    s=s.replace("*","")
    analysed_seq = ProteinAnalysis(s)
    analysed_seq2 = ProteinAnalysis(s2)
    Ph=analysed_seq2.charge_at_pH(7.2)-analysed_seq.charge_at_pH(7.2)
  else:
    Ph=-(ProteinAnalysis(s).charge_at_pH(7.2))
  return Ph
#Delta of hydrophoobicity
def dphysicoGr(s,s2):
  if len(s)>0 and len(s2)>0:
    s=s.replace("*","")
    analysed_seq = ProteinAnalysis(s)
    analysed_seq2 = ProteinAnalysis(s2)
    Gr=analysed_seq2.gravy()-analysed_seq.gravy()
  else: 
    Gr=-(ProteinAnalysis(s).gravy())
  return Gr
#Delta of Entropy
#https://www.genome.jp/entry/aaindex:HUTJ700102
#I    A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#   30.88   68.43   41.70   40.66   53.83   46.62   44.98   24.74   65.99   49.71
#   50.62   63.21   55.32   51.06   39.21   35.65   36.50   60.00   51.15   42.75
#//
def dentropycalc(s,s2):
  list1=[]
  list2=[]
  list1[:0]=s
  list2[:0]=s2
  out=0
  out1=0
  out2=0
  b_factor = {'A': '30.88', 'C': '53.83', 'E': '44.98', 'D': '40.66', 'G': '24.74', 'F': '51.06', 'I': '49.71', 'H': '65.99',
              'K': '63.21',
              'M': '55.32', 'L': '50.62','*':'0.0','N': '41.70', 'Q': '46.62', 'P': '53.83', 'S': '46.62', 'R': '68.43', 'T': '36.50', 'W': '60.00', 'V': '42.75','*':'0.0', 'Y': '51.15'}
  for i in list1:
    out1+=float(b_factor[i])
  for l in list2:
    out2+=float(b_factor[l])
  if len(s2) !=0:
    outp=(out2/len(s2))-(out1/len(s))
  else:
    outp=-(out1/len(s))
  return outp
#DDG calcuation
#https://www.genome.jp/entry/aaindex:YUTK870101
#
#The DDG value is calculated from the unfolding Gibbs free energy value of the mutated protein minus
#the unfolding Gibbs free energy value of the wild type (Kcal/mol).
#I    A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#     8.5      0.     8.2     8.5    11.0     6.3     8.8     7.1    10.1    16.8
#    15.0     7.9    13.3    11.2     8.2     7.4     8.8     9.9     8.8    12.0
#//
def ddG(s0,s2):
  list1=[]
  list1[:0]=s2
  out=0
  list0=[]
  list0[:0]=s0
  out0=0
  b_factor = {'A': '8.5', 'C': '11.0', 'E': '8.8', 'D': '8.5', 'G': '7.1', 'F': '11.2', 'I': '16.8', 'H': '10.1', 'K': '7.9',
              'M': '13.3', '*':'0.0','L': '15.0', 'N': '8.2', 'Q': '6.3', 'P': '8.2', 'S': '7.4', 'R': '0.0', 'T': '8.8', 'W': '9.9', 'V': '12.0', 'Y': '8.8'}
  for i in list1:
    out+=float(b_factor[i])
  for l in list0:
    out0+=float(b_factor[l])
  if len(s2) !=0:
    outp=(out/len(s2))-(out0/len(s0))
  else:
    outp=-(out0/len(s0))
  return outp
#https://www.genome.jp/entry/aaindex:TAKK010101
#   A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#     9.8     7.3     3.6     4.9     3.0     2.4     4.4       0    11.9    17.2
#    17.0    10.5    11.9    23.0    15.0     2.6     6.9    24.2    17.2    15.3
#//
#Delta of stability
def dstabcalc(s,s2):
  list1=[]
  list2=[]
  list1[:0]=s
  list2[:0]=s2
  out=0
  out1=0
  out2=0
  b_factor = {'A': '9.8', 'C': '3.0', 'E': '4.4', 'D': '4.9', 'G': '0.0', 'F': '23.0', 'I': '17.2', 'H': '11.9', 'K': '10.5',
              'M': '11.9', 'L': '17.0', 'N': '3.6', '*':'0.0','Q': '2.4', 'P': '15.0', 'S': '2.6', 'R': '7.3', 'T': '6.9', 'W': '24.2', 'V': '15.3', 'Y': '17.2'}
  for i in list1:
    out1+=float(b_factor[i])
  for l in list2:
    out2+=float(b_factor[l])
  if len(s2) !=0:
    outp=(out2/len(s2))-(out1/len(s))
  else:
    outp=-(out1/len(s))
  return outp
#delta of thermal capacity 
#https://www.genome.jp/entry/aaindex:HUTJ700101
#I    A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#   29.22   26.37   38.30   37.09   50.70   44.02   41.84   23.71   59.64   45.00
#   48.03   57.10   69.32   48.52   36.13   32.40   35.20   56.92   51.73   40.35
#//
def dthermalcapcalc(s,s2):
  list1=[]
  list2=[]
  list1[:0]=s
  list2[:0]=s2
  out=0
  out1=0
  out2=0
  b_factor = {'A': '29.22', 'C': '50.70', 'E': '41.84', 'D': '37.09', 'G': '23.71', 'F': '48.52', 'I': '45.00', 'H': '59.64', 'K': '57.10',
              'M': '69.32', 'L': '48.03', 'N': '38.30', '*':'0.0','Q': '44.02', 'P': '36.13', 'S': '32.40', 'R': '26.37', 'T': '35.20', 'W': '56.92', 'V': '40.35', 'Y': '51.73'}

  for i in list1:
    out1+=float(b_factor[i])
  for l in list2:
    out2+=float(b_factor[l])
  if len(s2) !=0:
    outp=(out2/len(s2))-(out1/len(s))
  else:
    outp=-(out1/len(s))
  return outp
#delta of volume change
#https://www.genome.jp/dbget-bin/www_bget?aaindex:BIGC670101
# A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#    52.6   109.1    75.7    68.4    68.3    89.7    84.7    36.3    91.9   102.0
#   102.0   105.1    97.7   113.9    73.6    54.9    71.2   135.4   116.2    85.1
def dVolcalc(s,s2):
  list1=[]
  list2=[]
  list1[:0]=s
  list2[:0]=s2
  out=0
  out1=0
  out2=0
  b_factor = {'A': '52.6', 'C': '68.3', 'E': '84.7', 'D': '68.4', 'G': '36.3','*':'0.0', 'F': '113.9', 'I': '102.0', 'H': '91.9', 'K': '105.1', 'M': '97.7', 'L': '102.0', 'N': '75.7', 'Q': '89.7', 'P': '73.6', 'S': '54.9', 'R': '109.1', 'T': '71.2', 'W': '36.3', 'V': '85.1', 'Y': '116.2'}
  for i in list1:
    out1+=float(b_factor[i])
  for l in list2:
    out2+=float(b_factor[l])
  if len(s2) !=0:
    outp=(out2/len(s2))-(out1/len(s))
  else:
    outp=-(out1/len(s))
  return outp

#delta of surface area
#https://www.genome.jp/entry/aaindex:JANJ780101
#I    A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#    27.8    94.7    60.1    60.6    15.5    68.7    68.2    24.5    50.7    22.8
#    27.6   103.0    33.5    25.5    51.5    42.0    45.0    34.7    55.2    23.7
#//

def dSAalc(s,s2):
  list1=[]
  list2=[]
  list1[:0]=s
  list2[:0]=s2
  out=0
  out1=0
  out2=0

  b_factor = {'A': '27.8', 'C': '15.5', 'E': '68.2', 'D': '60.6', 'G': '24.5', 'F': '25.5', 'I': '22.8', 'H': '50.7', 'K': '103.0', 'M': '33.5', 'L': '27.6',
              'N': '60.1', 'Q': '42.0', 'P': '51.5', 'S': '42.0', 'R': '94.7', '*':'0.0','T': '45.0', 'W': '34.7', 'V': '23.7', 'Y': '55.2'}
  for i in list1:
    out1+=float(b_factor[i])
  for l in list2:
    out2+=float(b_factor[l])
  if len(s2) !=0:
    outp=(out2/len(s2))-(out1/len(s))
  else:
    outp=-(out1/len(s))
  return outp

#delta of felxibilty
#https://www.genome.jp/dbget-bin/www_bget?aaindex:VINM940101
#I    A/L     R/K     N/M     D/F     C/P     Q/S     E/T     G/W     H/Y     I/V
#  0.984   1.008   1.048   1.068   0.906   1.037   1.094   1.031   0.950   0.927
#  0.935   1.102   0.952   0.915   1.049   1.046   0.997   0.904   0.929   0.931
#
#
def dFlexcalc(s,s2):
  list1=[]
  list2=[]
  list1[:0]=s
  list2[:0]=s2
  out=0
  out1=0
  out2=0
  b_factor = {'A': '0.984', 'C': '0.906', 'E': '1.094', 'D': '1.068', 'G': '1.031', 'F': '0.915', 'I': '0.927', 'H': '0.950', 'K': '1.102', 'M': '0.952', 'L': '0.935', 
              'N': '1.048', 'Q': '1.037', 'P': '1.049', 'S': '1.046','*':'0.0', 'R': '1.008', 'T': '0.997', 'W': '0.904', 'V': '0.931', 'Y': '0.929'}
  for i in list1:
    out1+=float(b_factor[i])
  for l in list2:
    out2+=float(b_factor[l])
  if len(s2) !=0:
    outp=(out2/len(s2))-(out1/len(s))
  else:
    outp=-(out1/len(s))
  return outp







65.4
0.984


In [ ]:
#Use this cell to define parameters for generating combiantorial library of protein s at specific positions in list x3
#Not for single site saturation mutagenesis
##Inputs Here 
import random
import sys
#never mind about the name
protein = set(["A", "R", "N", "D","C", "Q", "E", "G", "H", "I", "L","K","M" ,"F", "P", "S","T", "W", "Y", "V", "*"])
#put protein here

x3= [39, 40, 41,  54]
d =  len(x3)





In [ ]:
#Use this cell to define parameters for generating combiantorial library of protein s at specific positions in list x3
#Not for single site
%time

from itertools import combinations
output = sum([list(map(list, combinations(x3, i))) for i in range(len(x3) + 1)], [])
xxx=[]

while len(xxx) <= int(factt2(len(x3),len(x3)))+1:
  for i in output[1:]:
    xxx.extend(((mutate3(s,i))))


CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs
[[39], [40], [41], [54], [39, 40], [39, 41], [39, 54], [40, 41], [40, 54], [41, 54], [39, 40, 41], [39, 40, 54], [39, 41, 54], [40, 41, 54], [39, 40, 41, 54]]


In [ ]:
print(len(xxx))

194495


In [ ]:
#Use this cell to define parameters for generating singke position saturation mutagenisis library of protein s
import random
import sys
# nerver mind about the name
protein = set(["A", "R", "N", "D","C", "Q", "E", "G", "H", "I", "L","K","M" ,"F", "P", "S","T", "W", "Y", "V", "*"])

#put start of evolution index
x1=1 
#put end of evolution index
x2=len(s)
d=1

xxx=list(mutate2(s,d,x1,x2))


In [ ]:
#Compute Indepdenent contributions and first epistaitc contribtion method
# feel free to adapt
import pandas as pd
import numpy as np
aligner = Align.PairwiseAligner()
df = pd.DataFrame(data=pd.Series(xxx),columns=['Mutant'])
df.index = np.arange(1,len(df)+1)
df['Variants'] = df.apply(lambda row : gets(row['Mutant'],getss(s,row['Mutant'])), axis = 1)
df['Mutations'] = df.apply(lambda row : funxii(s,row['Mutant']), axis = 1)
df=df.drop_duplicates()
df['IndependentE'] = df.apply(lambda row : indp2(row['Mutations']), axis = 1)
df['EpistaticE1'] = df.apply(lambda row : epidmcalc2(row['Mutant'],getss(s,row['Mutant'])), axis = 1)
df

,Mutant,Variants,Mutations,IndependentE,EpistaticE1
1,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,C,A335C,-5.821086,1.774714
2,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,I,D139I,58.822261,64.359043
3,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,*,A189*,0.000000,0.856066
4,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,F,Y209F,20.839023,15.566505
5,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,H,P281H,-8.288362,3.844613
...,...,...,...,...,...
7097,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,N,D191N,-9.419384,2.982255
7098,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,M,D148M,-9.073409,1.329790
7099,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,G,D250G,-6.567807,5.018435
7100,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,R,G145R,6.884673,12.407458


In [ ]:
#Compute list of paramters
# feel free to adapt
df['EpistaticE2'] = df.apply(lambda row : epis2(row['Mutations']), axis = 1)
df['EpistaticE3'] = df.apply(lambda row : epis3(row['Mutations']), axis = 1)
df['TotalEffect'] = df.apply(lambda row : score0(row['IndependentE'],-row['EpistaticE1']), axis = 1)
df['TotalEffect2'] = df.apply(lambda row : score0(row['IndependentE'],row['EpistaticE2']), axis = 1)
df['TotalEffect3'] = df.apply(lambda row : score0(row['IndependentE'],row['EpistaticE3']), axis = 1)
df['Mutant1'] = df['Mutant']
df['Energy'] = df.apply(lambda row : sum(encoder2(row['Mutant1'],getss(s,row['Mutant1']))), axis = 1)

df['Mutant'] = df.apply(lambda row : xstop(row['Mutant']), axis = 1)
sums=sum(encoder(s))
df['EpistaticE4b'] = df.apply(lambda row : (row['Energy']-sum(encoder2(s,getss(s,row['Mutant1'])))), axis = 1)
df['TotalEffect4b'] = df.apply(lambda row : score0(row['IndependentE'],row['EpistaticE4b']), axis = 1)
df['DDG'] = df.apply(lambda row : ddG(s,row['Mutant']), axis = 1)
df['dMW'] = df.apply(lambda row : dphysicoMW(s,row['Mutant']), axis = 1)
df['dInstability_Index'] = df.apply(lambda row : dphysicoIns(s,row['Mutant']), axis = 1)
df['dThermal_Cap'] = df.apply(lambda row : dthermalcapcalc(s,row['Mutant']), axis = 1)
df['dEntropy'] = df.apply(lambda row : dentropycalc(s,row['Mutant']), axis = 1)
df['dStability'] = df.apply(lambda row : dstabcalc(s,row['Mutant']), axis = 1)
df['dCharge_pH7.2'] = df.apply(lambda row : dphysicoPh(s,row['Mutant']), axis = 1)
df['dHydrophobicity'] = df.apply(lambda row : dphysicoGr(s,row['Mutant']), axis = 1)
df['dVolume'] = df.apply(lambda row : dVolcalc(s,row['Mutant']), axis = 1)
df['dSurfaceArea'] = df.apply(lambda row : dSAalc(s,row['Mutant']), axis = 1)
df['dFlexibility'] = df.apply(lambda row : dFlexcalc(s,row['Mutant']), axis = 1)
df['MW'] = df.apply(lambda row : physicoMW(row['Mutant']), axis = 1)
df['Instability_Index'] = df.apply(lambda row : physicoIns(row['Mutant']), axis = 1)
df['Thermal_Cap'] = df.apply(lambda row : thermalcapcalc(row['Mutant']), axis = 1)
df['Entropy'] = df.apply(lambda row : entropycalc(row['Mutant']), axis = 1)
df['Stability'] = df.apply(lambda row : stabcalc(row['Mutant']), axis = 1)
df['Charge_pH7.2'] = df.apply(lambda row : physicoPh(row['Mutant']), axis = 1)
df['Hydrophobicity'] = df.apply(lambda row : physicoGr(row['Mutant']), axis = 1)
df['Volume'] = df.apply(lambda row : Volcalc(row['Mutant']), axis = 1)
df['SurfaceArea'] = df.apply(lambda row : SAalc(row['Mutant']), axis = 1)
df['Flexibility'] = df.apply(lambda row : Flexcalc(row['Mutant']), axis = 1)

df.to_csv("GB1a.csv")
df

,Mutant,Variants,Mutations,IndependentE,EpistaticE1,EpistaticE2,EpistaticE3,TotalEffect,TotalEffect2,TotalEffect3,Mutant1,Energy,EpistaticE4b,TotalEffect4b,DDG,dMW,dInstability_Index,dThermal_Cap,dEntropy,dStability,dCharge_pH7.2,dHydrophobicity,dVolume,dSurfaceArea,dFlexibility,MW,Instability_Index,Thermal_Cap,Entropy,Stability,Charge_pH7.2,Hydrophobicity,Volume,SurfaceArea,Flexibility
1,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,C,A335C,-5.821086,1.774714,-2.713572,-3.713568,-7.595800,-8.534658,-9.534654,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,1380.215845,-3.045620,-8.866706,0.007042,32.0650,-0.967324,0.060507,0.064648,-0.019155,-0.015602,0.001972,0.044225,-0.034648,-0.000220,39521.6640,24.632141,41.029183,46.345549,10.126197,-15.139418,-0.152113,81.781408,45.339437,0.999854
2,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,I,D139I,58.822261,64.359043,65.394830,-0.356675,-5.536783,124.217090,58.465586,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,1382.929199,-0.340710,58.481551,0.023380,-1.9298,0.000000,0.022282,0.025493,0.034648,0.999293,0.022535,0.094648,-0.106479,-0.000397,39487.6692,25.599465,40.990958,46.306394,10.180000,-14.124524,-0.131549,81.831831,45.267606,0.999676
3,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,*,A189*,0.000000,0.856066,0.000000,0.000000,-0.856066,0.000000,0.000000,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,1377.801071,-5.467540,-5.467540,-0.148057,-18763.0217,-1.835103,-0.727399,-0.233189,-0.571416,11.600337,-0.014533,-0.087715,0.639745,0.002390,20726.5773,23.764362,40.241277,46.047713,9.573936,-3.523480,-0.168617,81.649468,46.013830,1.002463
4,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,F,Y209F,20.839023,15.566505,27.346040,-1.188224,5.272518,48.185063,19.650799,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,1382.116332,-1.137520,19.701503,0.006761,-15.9994,-0.347606,-0.009042,-0.000254,0.016338,0.001582,0.011549,-0.006479,-0.083662,-0.000039,39473.5996,25.251859,40.959634,46.280648,10.161690,-15.122234,-0.142535,81.730704,45.290423,1.000034
5,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,H,P281H,-8.288362,3.844613,-2.655992,-4.557370,-12.132975,-10.944354,-12.845732,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,1379.039583,-4.226120,-12.514482,0.005352,40.0241,-1.085070,0.066225,0.034254,-0.008732,0.056832,-0.004507,0.051549,-0.002254,-0.000279,39529.6231,24.514394,41.034901,46.315155,10.136620,-15.066985,-0.158592,81.788732,45.371831,0.999794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7097,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,N,D191N,-9.419384,2.982255,-3.572154,-4.878994,-12.401638,-12.991538,-14.298377,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,1378.890019,-4.398030,-13.817414,-0.000845,-0.9848,0.000000,0.003408,0.002930,-0.003662,0.999293,0.000000,0.020563,-0.001408,-0.000056,39488.6142,25.599465,40.972085,46.283831,10.141690,-14.124524,-0.154085,81.757746,45.372676,1.000017
7098,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,M,D148M,-9.073409,1.329790,-3.762174,-4.762162,-10.403199,-12.835583,-13.835571,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,1379.211849,-4.034790,-13.108199,0.013521,16.1086,0.347606,0.090789,0.041296,0.019718,0.999293,0.015211,0.082535,-0.076338,-0.000327,39505.7076,25.947070,41.059465,46.322197,10.165070,-14.124524,-0.138873,81.819718,45.297746,0.999746
7099,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,G,D250G,-6.567807,5.018435,0.204209,-3.186350,-11.586241,-6.363597,-9.754157,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,1380.298801,-2.972060,-9.539867,-0.003944,-58.0361,-0.906197,-0.037690,-0.044845,-0.013803,0.999293,0.008732,-0.090423,-0.101690,-0.000104,39431.5629,24.693268,40.930986,46.236056,10.131549,-14.124524,-0.145352,81.646761,45.272394,0.999969
7100,MTKEKNVILTARDIVVEFDVRDKVLTAIRGVSLELVEGEVLALVGE...,R,G145R,6.884673,12.407458,13.570654,-2.484905,-5.522786,20.455327,4.

In [ ]:
#Some Scoring functions you can  optimize for your prefernces
def score0(s1,s2):
  s3=(s1)+s2
  return s3
def score1(s1,s2,s3,s4,s5,s6,s7,s8,s9,s10):
  s0=(-s1)+s2+s3+s4+s5+s6+s7+s8+s9+s10
  return s0
def score3(s1,s2,s3):
  s4=s1*(s2+s3)
  return s4
def score4(s1,s0,s2,s3):
  s4=(s1+s0)*(s2+s3)
  return s4
def score(s1,s2,s3):
  s4=s1+s2+s3
  return s4
def getss(s,s2):
  l=[]
  for i in range(0,len(s)-1):
    if (s[i]) != (s2[i]):
      l.append(i)
  l=(x+1 for x in l)
  return l if l!=[-1] and l!=[] else [0]

In [ ]:
#apply scoring functions to computed paramters
#we used some normalization functions in range of [0,0.3] and [0,1], feel free to adapt
df['dScore1'] = df.apply(lambda row :  score1(row['dInstability_Index'],row['dThermal_Cap'],row['dEntropy'],row['dCharge_pH7.2'],row['dHydrophobicity'],row['dVolume'],row['dSurfaceArea'],row['dFlexibility'],row['dStability'],row['dMW']), axis = 1)
min1=df['dScore1'].min()
max1=df['dScore1'].max()
def normad1(s):
  s0neo = (((s - min1) * (0.001 - 0.33)) / (max1 - min1)) + 0.001
  return s0neo
df['dScore1'] = df.apply(lambda row : normad1(row['dScore1']), axis = 1)

df['dScore0'] = df.apply(lambda row :  score0(row['TotalEffect'],row['DDG']), axis = 1)

min0=df['dScore0'].min()
max0=df['dScore0'].max()
def normad0(s):
  s0neo = (((s - min0) * (0.001 - 0.33)) / (max0 - min0)) + 0.001
  return s0neo
df['dScore0'] = df.apply(lambda row : normad0(row['dScore0']), axis = 1)


df['dScore2'] = df.apply(lambda row :  score4(row['Energy'],row['EpistaticE1'],row['dScore0'],row['dScore1']), axis = 1)

min2=df['dScore2'].min()
max2=df['dScore2'].max()
def normad2(s):
  s0neo = (((s - min2) * (0.001 - 0.33)) / (max2 - min2)) + 0.001
  return s0neo
df['dScore2'] = df.apply(lambda row : normad2(row['dScore2']), axis = 1)


df['Score1'] = df.apply(lambda row :  score1(row['Instability_Index'],row['Thermal_Cap'],row['Entropy'],row['Charge_pH7.2'],row['Hydrophobicity'],row['Volume'],row['SurfaceArea'],row['Flexibility'],row['Stability'],row['MW']), axis = 1)
min1=df['Score1'].min()
max1=df['Score1'].max()
def norma1(s):
  s0neo = (((s - min1) * (0.001 - 0.33)) / (max1 - min1)) + 0.001
  return s0neo
df['Score1'] = df.apply(lambda row : normad1(row['Score1']), axis = 1)


df['Score0'] = df.apply(lambda row :  score0(row['TotalEffect'],row['DDG']), axis = 1)
min0=df['Score0'].min()
max0=df['Score0'].max()
def normad0(s):
  s0neo = (((s - min0) * (0.001 - 0.33)) / (max0 - min0)) + 0.001
  return s0neo
df['Score0'] = df.apply(lambda row : normad0(row['Score0']), axis = 1)


df['Score2'] = df.apply(lambda row :  score4(row['Energy'],row['EpistaticE1'],row['Score0'],row['Score1']), axis = 1)
min2=df['Score2'].min()
max2=df['Score2'].max()
def normad2(s):
  s0neo = (((s - min2) * (0.001 - 0.33)) / (max2 - min2)) + 0.001
  return s0neo
df['Score2'] = df.apply(lambda row : normad2(row['Score2']), axis = 1)


def scorefinal(s0,s1,s2):
  
  s0neo = (((s0 - min0) * (0.1 - 0.33)) / (max0 - min0)) + 0.1
  s1neo = (((s1 - min1) * (0.1- 0.33)) / (max1 - min1)) + 0.1
  s2neo = (((s2 - min2) * (0.1 - 0.33)) / (max2 - min2)) + 0.1
  s3=s0neo+s1neo+s2neo
  return s3
df['dScore'] = df.apply(lambda row :  scorefinal(row['dScore0'],row['dScore1'],row['dScore2']), axis = 1)
df['tScore'] = df.apply(lambda row :  scorefinal(row['Score0'],row['Score1'],row['Score2']), axis = 1)

#Use these values to plot a landscape if relevant
xaxis1=np.array(df['dScore1'].tolist())
yaxis1=np.array(df['dScore0'].tolist())
zaxis1=np.array(df['dScore2'].tolist())

#Use these values to plot a landscape if relevant
xaxis2=np.array(df['Score1'].tolist())
yaxis2=np.array(df['Score0'].tolist())
zaxis2=np.array(df['Score2'].tolist())

In [ ]:
#Concatenate Greg1 PCA indices
df3=pd.DataFrame(columns=createList(0, 18))
df3 = df.apply(lambda row : Greg1(row['Mutant']), axis = 1)
df4=pd.DataFrame(data=df3.tolist())
frames=[df4,df]
df5 = pd.concat(frames,axis=1)
df5=df5.dropna()
df5

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,Mutant,Variants,Mutations,IndependentE,EpistaticE1,EpistaticE2,EpistaticE3,TotalEffect,TotalEffect2,TotalEffect3,Energy,Mutant1,EpistaticE4b,TotalEffect4b,DDG,dMW,dInstability_Index,dThermal_Cap,dEntropy,dStability,dCharge_pH7.2,dHydrophobicity,dVolume,dSurfaceArea,dFlexibility,MW,Instability_Index,Thermal_Cap,Entropy,Stability,Charge_pH7.2,Hydrophobicity,Volume,SurfaceArea,Flexibility
1,-0.501429,0.206429,-0.259821,0.847143,0.515000,0.129107,0.445714,0.456786,0.030893,0.513929,0.824286,1.088036,0.748393,0.138214,-0.094821,-0.584643,0.685179,-1.010357,-0.238750,MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGADGEWTYD...,ADGV,V39A,-6.033506,13.739755,-3.077537,-4.077532,-19.773262,-9.111044,-10.111038,192.197051,MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGADGEWTYD...,-3.036567,-9.070073,-0.062500,-28.0531,-3.482143e-01,-0.198750,-0.211964,-0.098214,0.000000,-0.042857,-0.580357,0.073214,0.000946,6167.6830,2.235714,40.560000,42.794821,9.196429,-4.611430,-0.676786,78.271429,48.810714,1.010357
2,-0.567500,0.116964,-0.303036,0.849643,0.382321,0.067679,0.450536,0.443036,-0.044107,0.489107,0.845714,1.008393,0.648036,0.110357,-0.126607,-0.602857,0.675893,-1.042500,-0.279643,MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGFDGEWTYD...,FDGV,V39F,-4.127134,13.579607,-4.077537,-16.118096,-17.706742,-8.204672,-20.245230,191.398930,MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGFDGEWTYD...,-3.834687,-7.961822,-0.014286,48.0428,4.887500e+00,0.145893,0.148393,0.137500,0.000000,-0.025000,0.514286,0.032143,-0.000286,6243.7789,7.471429,40.904643,43.155179,9.432143,-4.611430,-0.658929,79.366071,48.769643,1.009125
3,-0.626607,0.220536,-0.267857,0.874286,0.540179,0.101964,0.520000,0.463929,-0.048750,0.471250,0.938929,1.023750,0.679286,0.139107,-0.181429,-0.575357,0.690179,-1.055893,-0.256071,MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGPDGEWTYD...,PDGV,V39P,-2.079442,13.579607,-4.077537,-16.118096,-15.659049,-6.156979,-18.197537,191.346104,MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGPDGEWTYD...,-3.887514,-5.966955,-0.067857,-2.0158,1.337500e+00,-0.075357,0.197857,-0.005357,0.000000,-0.103571,-0.205357,0.496429,0.002107,6193.7203,3.921429,40.683393,43.204643,9.289286,-4.611430,-0.737500,78.646429,49.233929,1.011518
4,-0.501786,0.200893,-0.298036,0.871964,0.529821,0.079107,0.444643,0.525714,-0.028750,0.536250,0.850179,0.995000,0.636429,0.085714,-0.164821,-0.502321,0.700536,-0.998750,-0.355000,MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGKDGEWTYD...,KDGV,V39K,-6.084499,13.579607,-4.077537,-16.118096,-19.664107,-10.162037,-22.202595,191.359482,MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGKDGEWTYD...,-3.874135,-9.958635,-0.073214,29.0413,1.167857e+00,0.299107,0.365357,-0.085714,0.998418,-0.144643,0.357143,1.416071,0.003054,6224.7774,3.751786,41.057857,43.372143,9.208929,-3.613012,-0.778571,79.208929,50.153571,1.012464
5,-0.602500,0.226786,-0.317321,0.820357,0.532321,0.089464,0.452857,0.501786,-0.078393,0.471071,0.846786,1.020000,0.636964,0.079464,-0.021964,-0.607500,0.721607,-0.958750,-0.263929,MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGIDGEWTYD...,IDGV,V39I,3.953353,14.125194,6.922463,-1.679642,-10.171841,10.875815,2.273711,193.846713,MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGIDGEWTYD...,-1.386905,2.566448,0.085714,14.0266,1.167857e+00,0.083036,0.124286,0.033929,0.000000,0.005357,0.301786,-0.016071,-0.000071,6209.7627,3.751786,40.841786,43.131071,9.328571,-4.611430,-0.628571,79.153571,48.721429,1.009339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194476,-0.471509,0.172830,-0.133585,0.813019,0.539245,0.001698,0.473774,0.335094,-0.066792,0.487736,0.813208,0.976981,0.910755,0.064528,-0.130000,-0.652453,0.610755,-0.828491,-0.346038,MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGDQYEWTYD...,DQYR,"V39D, D40Q, G41Y, V54R",-17.390461,1.613549,-15.726583,-52.588387,-19.004010,-33.117044,-69.978848,180.102857,MTYKLILNGKTLK